In [41]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import os
import re
import random

class MaskedLMEvaluator:
    def __init__(self):
        # Load the models and tokenizers
        self.models = {
            "kaz_roberta": self.load_model("kz-transformers/kaz-roberta-conversational"),
            "mbert": self.load_model("bert-base-multilingual-cased")
        }
        self.random_seed = 42

    def set_random_seed(self, seed):
        self.random_seed = seed
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

    def load_model(self, model_name):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForMaskedLM.from_pretrained(model_name)
        model.eval()
        return model, tokenizer

    def load_all_texts(self, directory):
        texts = []
        for file_name in os.listdir(directory):
            if file_name.endswith(".txt"):
                with open(os.path.join(directory, file_name), 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                    lines = [line.strip() for line in lines if len(line.strip()) > 10]
                    texts.extend(lines)
        return texts

    def evaluate_model(self, texts, model_name, num_samples=100):
        model, tokenizer = self.models[model_name]
        mask_token = "<mask>" if model_name == "kaz_roberta" else "[MASK]"

        # Set random seed for consistency
        self.set_random_seed(self.random_seed)

        # Convert texts to lowercase, shuffle, and extract first sentence
        texts = [text.lower() for text in texts]
        random.shuffle(texts)
        texts = [self.get_first_sentence(text) for text in texts]

        y_true = []
        y_pred = []
        results = []

        for text in tqdm(texts[:num_samples]):
            words = text.split()
            if len(words) < 3:
                continue

            # Select a word longer than 1 character to mask
            valid_indices = [i for i in range(1, len(words) - 1) if len(words[i]) > 1]
            if not valid_indices:
                continue

            idx = random.choice(valid_indices)
            original_word = words[idx]

            # Mask the selected word
            masked_sentence = (
                " ".join(words[:idx]) + f" {mask_token} " + " ".join(words[idx + 1:])
            )

            if mask_token not in masked_sentence:
                continue

            predicted_word = self.predict_with_model(model, tokenizer, masked_sentence, mask_token)
            if not predicted_word:
                continue

            y_true.append(original_word)
            y_pred.append(predicted_word)

            results.append({
                "original_text": text,
                "masked_text": masked_sentence,
                "original_masked_word": original_word,
                "mask_predicted": predicted_word
            })

        # Create a DataFrame
        df_results = pd.DataFrame(results)

        # Calculate Metrics
        if not df_results.empty:
            accuracy = accuracy_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
            recall = recall_score(y_true, y_pred, average="micro", zero_division=0)
            f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)
            jaccard = np.mean(
                [self.jaccard_similarity(y_true[i], y_pred[i]) for i in range(len(y_true))]
            )

            print(f"Model: {model_name}")
            print(f"Accuracy: {accuracy:.4f}")
            # print(f"Precision: {precision:.4f}")
            # print(f"Recall: {recall:.4f}")
            # print(f"F1 Score: {f1:.4f}")
            print(f"Average Jaccard Similarity: {jaccard:.4f}")

        return df_results

    def predict_with_model(self, model, tokenizer, masked_sentence, mask_token):
        try:
            inputs = tokenizer(masked_sentence, return_tensors="pt", truncation=True, max_length=512)
            mask_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

            if len(mask_index) == 0:
                return ""

            with torch.no_grad():
                logits = model(**inputs).logits

            mask_logits = logits[0, mask_index, :]
            predicted_token_id = torch.argmax(mask_logits, dim=1).item()

            # Decode the predicted token
            predicted_word = tokenizer.decode([predicted_token_id]).strip()
            return predicted_word
        except Exception as e:
            print(f"Error during prediction: {e}")
            return ""

    def jaccard_similarity(self, word1, word2):
        set1 = set(word1)
        set2 = set(word2)
        if not set1 or not set2:
            return 0.0
        return len(set1.intersection(set2)) / len(set1.union(set2))

    def get_first_sentence(self, text):
        sentences = re.split(r"[.!?]", text)
        return sentences[0].strip() if sentences else text.strip()


In [42]:

evaluator = MaskedLMEvaluator()

# Load texts
texts = evaluator.load_all_texts("../data/processed")

# Set random seed for consistency across models
evaluator.set_random_seed(42)

# Evaluate models
df_kaz_roberta = evaluator.evaluate_model(texts, model_name="kaz_roberta", num_samples=3000)
df_mbert = evaluator.evaluate_model(texts, model_name="mbert", num_samples=3000)

# Save DataFrames
if not df_kaz_roberta.empty:
    df_kaz_roberta.to_csv("kaz_roberta_results.csv", index=False)

if not df_mbert.empty:
    df_mbert.to_csv("mbert_results.csv", index=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 3000/3000 [06:41<00:00,  7.47it/s]


Model: kaz_roberta
Accuracy: 0.1345
Average Jaccard Similarity: 0.2691


100%|██████████| 3000/3000 [18:00<00:00,  2.78it/s]


Model: mbert
Accuracy: 0.0268
Average Jaccard Similarity: 0.1064


In [43]:
df_mbert

,original_text,masked_text,original_masked_word,mask_predicted
0,"қайтқан қаздай тіркеліп, сыңсып көшіп бара жат...","қайтқан қаздай тіркеліп, сыңсып көшіп [MASK] ж...",бара,","
1,осы атасынан бастап оспан-қожа өзімен үшінші б...,осы атасынан бастап оспан-қожа [MASK] үшінші б...,өзімен,##ң
2,ол мені түсінгендей болды,ол мені [MASK] болды,түсінгендей,де
3,"иә, алтайды асып, шыңғысханның мың сан әскері ...","иә, алтайды асып, шыңғысханның мың сан әскері ...",мынау,да
4,ендеше он алты жасымнан отыз алты жасыма дейін...,ендеше он алты жасымнан отыз алты жасыма дейін...,жиырма,","
...,...,...,...,...
2945,"мынау абай көзіне көрініп тұрған, кесіп алғанд...","мынау абай көзіне көрініп тұрған, кесіп алғанд...","қалған,",жатқан
2946,мына ошақтардың астында жылтыраған оттардың ар...,мына [MASK] астында жылтыраған оттардың арасы ...,ошақтардың,жер
2947,"кітап сатып алу арқылы сіз өзіңіздің де, елімі...","кітап сатып алу арқылы сіз өзіңіздің де, елімі...",де,де
2948,"жуас жігіт сәлмен өздіпмен дедек қағып, таңырқ...","жуас жігіт сәлмен өздіпмен дедек қағып, таңырқ...","та,",","


In [44]:
df_kaz_roberta

,original_text,masked_text,original_masked_word,mask_predicted
0,"қайтқан қаздай тіркеліп, сыңсып көшіп бара жат...","қайтқан қаздай тіркеліп, сыңсып көшіп <mask> ж...",бара,бара
1,осы атасынан бастап оспан-қожа өзімен үшінші б...,осы атасынан бастап оспан-қожа <mask> үшінші б...,өзімен,бастаған
2,ол мені түсінгендей болды,ол мені <mask> болды,түсінгендей,түсінетін
3,"иә, алтайды асып, шыңғысханның мың сан әскері ...","иә, алтайды асып, шыңғысханның мың сан әскері ...",мынау,","
4,ендеше он алты жасымнан отыз алты жасыма дейін...,ендеше он алты жасымнан отыз алты жасыма дейін...,жиырма,","
...,...,...,...,...
2946,"мынау абай көзіне көрініп тұрған, кесіп алғанд...","мынау абай көзіне көрініп тұрған, кесіп алғанд...","қалған,",","
2947,мына ошақтардың астында жылтыраған оттардың ар...,мына <mask> астында жылтыраған оттардың арасы ...,ошақтардың,судың
2948,"кітап сатып алу арқылы сіз өзіңіздің де, елімі...","кітап сатып алу арқылы сіз өзіңіздің де, елімі...",де,де
2949,"жуас жігіт сәлмен өздіпмен дедек қағып, таңырқ...","жуас жігіт сәлмен өздіпмен дедек қағып, таңырқ...","та,",","
